# Basic Impact Evaluation

This notebook demonstrates the basic workflow for evaluating causal impact using the **Impact Engine**.

## Workflow Overview

1. Generate product characteristics using the catalog simulator
2. Configure the impact engine
3. Run impact evaluation
4. Review results

## Setup

Import the required packages.

In [ ]:
import json
import os

import yaml
from impact_engine import evaluate_impact
from online_retail_simulator import simulate

## Step 1: Generate Product Characteristics

First, we generate a synthetic product catalog using the **Online Retail Simulator**.

In [ ]:
# Run simulation using config file
os.makedirs("output/demo_basic", exist_ok=True)

job_info = simulate("configs/demo_basic_catalog.yaml")

# Save products to CSV
products = job_info.load_df("products")
products_path = "output/demo_basic/products.csv"
products.to_csv(products_path, index=False)

print(f"Generated {len(products)} products")
print(f"Saved to: {products_path}")
products.head()

## Step 2: Configure Impact Engine

Create a configuration file that specifies:
- **DATA**: Where to get products and how to simulate metrics
- **MEASUREMENT**: Which statistical model to use

In [3]:
# Impact engine configuration
impact_config = {
    "DATA": {
        "path": products_path,
        "SOURCE": {
            "type": "simulator",
            "CONFIG": {
                "mode": "rule",
                "seed": 42,
                "path": products_path,
                "start_date": "2024-11-01",
                "end_date": "2024-11-30",
            },
        },
        "TRANSFORM": {"FUNCTION": "aggregate_by_date", "PARAMS": {"metric": "revenue"}},
    },
    "MEASUREMENT": {
        "MODEL": "interrupted_time_series",
        "PARAMS": {"intervention_date": "2024-11-15", "dependent_variable": "revenue"},
    },
    "OUTPUT": {"PATH": "output/demo_basic"},
}

# Save configuration
config_path = "output/demo_basic/config.yaml"
with open(config_path, "w") as f:
    yaml.dump(impact_config, f, default_flow_style=False)

print(f"Configuration saved to: {config_path}")
print("\nConfiguration:")
print(yaml.dump(impact_config, default_flow_style=False))

Configuration saved to: output/demo_basic/config.yaml

Configuration:
DATA:
  SOURCE:
    CONFIG:
      end_date: '2024-11-30'
      mode: rule
      path: output/demo_basic/products.csv
      seed: 42
      start_date: '2024-11-01'
    type: simulator
  TRANSFORM:
    FUNCTION: aggregate_by_date
    PARAMS:
      metric: revenue
  path: output/demo_basic/products.csv
MEASUREMENT:
  MODEL: interrupted_time_series
  PARAMS:
    dependent_variable: revenue
    intervention_date: '2024-11-15'
OUTPUT:
  PATH: output/demo_basic



## Step 3: Run Impact Evaluation

Call `evaluate_impact()` with the configuration file. The engine handles:
- Loading products
- Simulating daily metrics
- Aggregating data
- Running the interrupted time series model

In [4]:
# Run impact evaluation
result_path = evaluate_impact(config_path)
print(f"Results saved to: {result_path}")

Results saved to: /home/peisenha/office/business/eisenhauer-io/tools/impact-engine/documentation/notebooks/data/job-impact-engine-20260124-064958-bbb998e7/results/impact_results.json


## Step 4: Review Results

Load and display the impact evaluation results.

In [ ]:
# Load results
with open(result_path, "r") as f:
    results = json.load(f)

data = results["data"]
model_params = data["model_params"]
estimates = data["impact_estimates"]
summary = data["model_summary"]

print("=" * 50)
print("IMPACT EVALUATION RESULTS")
print("=" * 50)
print(f"\nModel: {results['model_type']}")
print(f"Intervention Date: {model_params['intervention_date']}")
print(f"Dependent Variable: {model_params['dependent_variable']}")

print("\n--- Impact Estimates ---")
print(f"Pre-intervention mean:  ${estimates['pre_intervention_mean']:,.2f}")
print(f"Post-intervention mean: ${estimates['post_intervention_mean']:,.2f}")
print(f"Absolute change:        ${estimates['absolute_change']:,.2f}")
print(f"Percent change:         {estimates['percent_change']:.1f}%")

print("\n--- Model Summary ---")
print(f"Observations:   {summary['n_observations']}")
print(f"Pre-period:     {summary['pre_period_length']} days")
print(f"Post-period:    {summary['post_period_length']} days")